In [1]:
import pandas as pd
import boto
import s3fs
import pandas_profiling
import matplotlib.pyplot as plt
import datetime 

In [2]:
df = pd.read_csv('s3://g13batalha/2018-11-10_Sistema_E-Saude_Medicos_-_Base_de_Dados.csv',
                 sep=';', error_bad_lines=False, low_memory=False, encoding='windows_1252')

In [3]:
df.head()

,Data do Atendimento,Data de Nascimento,Sexo,Código do Tipo de Unidade,Tipo de Unidade,Código da Unidade,Descrição da Unidade,Código do Procedimento,Descrição do Procedimento,Código do CBO,...,Tipo de Habitação,Destino Lixo,Fezes/Urina,Cômodos,Em Caso de Doença,Grupo Comunitário,Meio de Comunicacao,Meio de Transporte,Municício,Bairro
0,18/09/2018 20:39:46,12/03/2011 00:00:00,F,2,UPA,17396,UPA BOA VISTA,301060096,ATENDIMENTO MEDICO EM UNIDADE DE PRONTO ATENDI...,225125,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PORTO AMAZONAS,BAIRRO NAO INFORMADO ...
1,30/09/2018 18:38:45,23/09/1984 00:00:00,F,2,UPA,5323495,UPA CIDADE INDUSTRIAL,301060029,ATENDIMENTO DE URGENCIA C/ OBSERVACAO ATE 24 H...,225125,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CURITIBA,CIDADE INDUSTRIAL
2,20/09/2018 19:26:16,03/12/1996 00:00:00,F,2,UPA,17396,UPA BOA VISTA,301060096,ATENDIMENTO MEDICO EM UNIDADE DE PRONTO ATENDI...,225125,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CURITIBA,CIDADE INDUSTRIAL
3,05/07/2018 12:56:28,13/06/1931 00:00:00,M,1,BASICO,16845,UMS SAO BRAZ,301010064,CONSULTA MEDICA EM ATENCAO BASICA,225125,...,TIJOLO/ALVENARIA COM REVESTIMENTO,COLETADO,SISTEMA DE ESGOTO,5.0,UNIDADE DE SAUDE,NAO PARTICIPA,TELEVISAO,ONIBUS,CURITIBA,SAO BRAZ ...
4,26/07/2018 07:46:41,13/06/1931 00:00:00,M,1,BASICO,16845,UMS SAO BRAZ,301010064,CONSULTA MEDICA EM ATENCAO BASICA,225125,...,TIJOLO/ALVENARIA COM REVESTIMENTO,COLETADO,SISTEMA DE ESGOTO,5.0,UNIDADE DE SAUDE,NAO PARTICIPA,TELEVISAO,ONIBUS,CURITIBA,SAO BRAZ ...


In [4]:
df.shape

(962710, 37)

In [5]:
df.columns

Index(['Data do Atendimento', 'Data de Nascimento', 'Sexo',
       'Código do Tipo de Unidade', 'Tipo de Unidade', 'Código da Unidade',
       'Descrição da Unidade', 'Código do Procedimento',
       'Descrição do Procedimento', 'Código do CBO', 'Descrição do CBO',
       'Código do CID', 'Descrição do CID', 'Solicitação de Exames',
       'Qtde Prescrita Farmácia Curitibana',
       'Qtde Dispensada Farmácia Curitibana',
       'Qtde de Medicamento Não Padronizado',
       'Encaminhamento para Atendimento Especialista', 'Área de Atuação',
       'Desencadeou Internamento', 'Data do Internamento',
       'Estabelecimento Solicitante', 'Estabelecimento Destino',
       'CID do Internamento', 'Tratamento no Domicílio', 'Abastecimento',
       'Energia Elétrica', 'Tipo de Habitação', 'Destino Lixo', 'Fezes/Urina',
       'Cômodos', 'Em Caso de Doença', 'Grupo Comunitário',
       'Meio de Comunicacao', 'Meio de Transporte', 'Municício', 'Bairro'],
      dtype='object')

In [6]:
columns = [
    'Data do Atendimento', 'Descrição da Unidade', 
    'Desencadeou Internamento', 'Energia Elétrica', 
    'Tipo de Habitação', 'Destino Lixo', 'Fezes/Urina',
    'Meio de Transporte', 'Bairro']

In [7]:
df = df[columns]

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 962710 entries, 0 to 962709
Data columns (total 9 columns):
Data do Atendimento         962710 non-null object
Descrição da Unidade        962710 non-null object
Desencadeou Internamento    962710 non-null object
Energia Elétrica            962710 non-null object
Tipo de Habitação           847016 non-null object
Destino Lixo                847014 non-null object
Fezes/Urina                 847015 non-null object
Meio de Transporte          846958 non-null object
Bairro                      962710 non-null object
dtypes: object(9)
memory usage: 66.1+ MB


In [9]:
df.isnull().sum()

Data do Atendimento              0
Descrição da Unidade             0
Desencadeou Internamento         0
Energia Elétrica                 0
Tipo de Habitação           115694
Destino Lixo                115696
Fezes/Urina                 115695
Meio de Transporte          115752
Bairro                           0
dtype: int64

In [10]:
df = df.dropna()

# Create Target Value

In [11]:
df['Data do Atendimento'] = pd.to_datetime(df['Data do Atendimento'])
df['Data_Dia'] = df['Data do Atendimento'].dt.floor('d')
df['Data_Hora'] = df['Data do Atendimento'].dt.floor('h')

In [12]:
df_target = df.groupby(['Data_Dia', 'Descrição da Unidade']).agg('count').reset_index()

In [13]:
df_target = df_target.iloc[:, :3]

In [14]:
df_target.columns = ['Data_Dia', 'Descrição da Unidade', 'Atendimentos']

In [15]:
df_target.head()

,Data_Dia,Descrição da Unidade,Atendimentos
0,2018-01-07,UPA BOA VISTA,272
1,2018-01-07,UPA BOQUEIRAO,315
2,2018-01-07,UPA CAJURU,336
3,2018-01-07,UPA CAMPO COMPRIDO,273
4,2018-01-07,UPA FAZENDINHA,277


# Groupings

### Tipo Habitação

In [16]:
df['Tipo de Habitação'].value_counts()

TIJOLO/ALVENARIA COM REVESTIMENTO    740635
MADEIRA APARELHADA                    75331
TIJOLO ALVENARIA SEM REVESTIMENTO     14662
OUTRO MATERIAL                        11577
MATERIAL APROVEITADO                   2429
TAIPA COM REVESTIMENTO                 1762
TAIPA SEM REVESTIMENTO                  374
PALHA                                   178
Name: Tipo de Habitação, dtype: int64

In [17]:
df['Habitacao_ma_qualidade'] = 0
df.loc[df['Tipo de Habitação'] != 'TIJOLO/ALVENARIA COM REVESTIMENTO', 'Habitacao_ma_qualidade'] = 1

In [18]:
df['Habitacao_ma_qualidade'].value_counts()

0    740635
1    106313
Name: Habitacao_ma_qualidade, dtype: int64

### Destino Lixo

In [19]:
df['Destino Lixo'].value_counts()

COLETADO              837723
OUTROS                  6640
QUEIMADO/ENTERRADO      1467
CÉU ABERTO              1118
Name: Destino Lixo, dtype: int64

In [20]:
df['Lixo_ma_qualidade'] = 0
df.loc[df['Destino Lixo'] != 'COLETADO', 'Lixo_ma_qualidade'] = 1

In [21]:
df['Lixo_ma_qualidade'].value_counts()

0    837723
1      9225
Name: Lixo_ma_qualidade, dtype: int64

### Energia Elétrica

In [22]:
df['Energia Elétrica'].value_counts()

Sim    804402
Nao     42546
Name: Energia Elétrica, dtype: int64

In [23]:
df['Sem_energia'] = 0
df.loc[df['Energia Elétrica'] != 'Sim', 'Sem_energia'] = 1

### Meio de Transporte 

In [24]:
df['Sem_Carro'] = df['Meio de Transporte'].apply(lambda x: 'CARRO' not in x)

In [25]:
df['Sem_Carro'].value_counts()

True     521912
False    325036
Name: Sem_Carro, dtype: int64

# Choose new columns

In [26]:
df.columns

Index(['Data do Atendimento', 'Descrição da Unidade',
       'Desencadeou Internamento', 'Energia Elétrica', 'Tipo de Habitação',
       'Destino Lixo', 'Fezes/Urina', 'Meio de Transporte', 'Bairro',
       'Data_Dia', 'Data_Hora', 'Habitacao_ma_qualidade', 'Lixo_ma_qualidade',
       'Sem_energia', 'Sem_Carro'],
      dtype='object')

In [27]:
columns = [
    'Data_Dia', 'Descrição da Unidade',
    'Desencadeou Internamento',
    'Habitacao_ma_qualidade', 'Lixo_ma_qualidade',
    'Sem_energia', 'Sem_Carro'
]

In [28]:
df = df[columns]

In [29]:
df.head()

,Data_Dia,Descrição da Unidade,Desencadeou Internamento,Habitacao_ma_qualidade,Lixo_ma_qualidade,Sem_energia,Sem_Carro
3,2018-05-07,UMS SAO BRAZ,Nao,0,0,0,True
4,2018-07-26,UMS SAO BRAZ,Nao,0,0,0,True
5,2018-10-08,UPA TATUQUARA,Nao,1,0,0,True
7,2018-07-16,UMS TARUMA PSF,Nao,0,0,0,False
8,2018-12-07,UMS TARUMA PSF,Nao,0,0,0,False


In [30]:
df.shape

(846948, 7)

In [31]:
df = df.sort_values('Data_Dia')
df_grouped = df.set_index('Data_Dia')
df_grouped.head()

,Descrição da Unidade,Desencadeou Internamento,Habitacao_ma_qualidade,Lixo_ma_qualidade,Sem_energia,Sem_Carro
Data_Dia,,,,,,
2018-01-07,UPA TATUQUARA,Nao,1,0,0,False
2018-01-07,UPA BOA VISTA,Nao,0,0,0,False
2018-01-07,UPA SITIO CERCADO,Nao,0,0,0,True
2018-01-07,UPA SITIO CERCADO,Nao,0,0,0,True
2018-01-07,UPA BOQUEIRAO,Nao,0,0,1,True


In [32]:
# Missing days should be threated in a future version
df_final = df_grouped.groupby(['Descrição da Unidade', 'Data_Dia']).agg('sum').rolling(7).mean().reset_index()

### Drop 7 first rows because of rolling window

In [33]:
df_final = df_final.dropna()

In [34]:
df_final.sort_values('Data_Dia').head()

,Descrição da Unidade,Data_Dia,Habitacao_ma_qualidade,Lixo_ma_qualidade,Sem_energia,Sem_Carro
8361,UPA TATUQUARA,2018-01-07,45.285714,4.428571,22.428571,250.285714
7755,UPA BOQUEIRAO,2018-01-07,44.857143,3.857143,15.714286,148.714286
7941,UPA CAMPO COMPRIDO,2018-01-07,36.000000,3.000000,19.142857,220.571429
7848,UPA CAJURU,2018-01-07,42.571429,5.142857,10.714286,172.000000
8175,UPA PINHEIRINHO,2018-01-07,28.428571,2.857143,13.857143,176.571429


In [35]:
df_final['weekday'] = df_final['Data_Dia'].dt.dayofweek

In [36]:
df_final.shape

(8448, 7)

In [37]:
df_final.head()

,Descrição da Unidade,Data_Dia,Habitacao_ma_qualidade,Lixo_ma_qualidade,Sem_energia,Sem_Carro,weekday
6,CENTRO DE ESPECIALIDADES MAE CURITIBANA,2018-04-07,10.857143,2.285714,6.428571,58.571429,5
7,CENTRO DE ESPECIALIDADES MAE CURITIBANA,2018-04-09,12.285714,1.714286,7.142857,67.428571,0
8,CENTRO DE ESPECIALIDADES MAE CURITIBANA,2018-05-07,11.285714,1.285714,7.142857,70.857143,0
9,CENTRO DE ESPECIALIDADES MAE CURITIBANA,2018-05-09,14.428571,1.285714,6.428571,76.000000,2
10,CENTRO DE ESPECIALIDADES MAE CURITIBANA,2018-06-07,12.714286,0.714286,6.000000,68.857143,3


# Add Lat Long

In [38]:
df_lat_long = pd.read_csv('lat_long.csv')
df_lat_long.head()

,Descrição da Unidade,Lat,Long
0,CENTRO DE ESPECIALIDADES MAE CURITIBANA,-25.424009,-49.276112
1,CENTRO DE ESPECIALIDADES MEDICAS MATRIZ,-25.433020,-49.271729
2,CENTRO DE ESPECIALIDADES OUVIDOR PARDINHO,-25.443760,-49.271271
3,CENTRO DE ESPECIALIDADES SALGADO FILHO,-25.488461,-49.223061
4,CENTRO DE ESPECIALIDADES SANTA FELICIDADE,-25.412571,-49.350670


In [39]:
df_final = df_final.merge(df_lat_long)

# Add Temperature

In [40]:
df_temp = pd.read_csv('s3://g13batalha/temp_JUL_NOV_2018.csv',
                      sep=',', error_bad_lines=False, low_memory=False, encoding='windows_1252')

In [41]:
df_temp.columns = ['Data_Dia', 'temp_max', 'temp_min']

In [42]:
df_temp.head()

,Data_Dia,temp_max,temp_min
0,01/07/2018,23,13
1,02/07/2018,24,11
2,03/07/2018,20,10
3,04/07/2018,19,10
4,05/07/2018,24,10


In [43]:
df_temp['Data_Dia'] = pd.to_datetime(df_temp['Data_Dia'])

In [44]:
df_final = df_final.merge(df_temp)

# Merge Target Value

In [45]:
df_final.shape

(8448, 11)

In [46]:
df_target.shape

(8454, 3)

In [47]:
df_final = df_final.merge(df_target, on=['Data_Dia', 'Descrição da Unidade'])

In [48]:
df_model = df_final

In [49]:
df_model.shape

(8448, 12)

In [50]:
df_model.head()

,Descrição da Unidade,Data_Dia,Habitacao_ma_qualidade,Lixo_ma_qualidade,Sem_energia,Sem_Carro,weekday,Lat,Long,temp_max,temp_min,Atendimentos
0,CENTRO DE ESPECIALIDADES MAE CURITIBANA,2018-04-07,10.857143,2.285714,6.428571,58.571429,5,-25.424009,-49.276112,19,10,96
1,CENTRO DE ESPECIALIDADES MEDICAS MATRIZ,2018-04-07,10.857143,0.571429,2.285714,50.714286,5,-25.433020,-49.271729,19,10,77
2,CENTRO DE ESPECIALIDADES OUVIDOR PARDINHO,2018-04-07,3.857143,0.000000,1.000000,16.000000,5,-25.443760,-49.271271,19,10,7
3,CENTRO DE ESPECIALIDADES SALGADO FILHO,2018-04-07,3.428571,0.428571,1.857143,23.571429,5,-25.488461,-49.223061,19,10,36
4,CENTRO DE ESPECIALIDADES SANTA FELICIDADE,2018-04-07,21.714286,2.142857,9.142857,106.428571,5,-25.412571,-49.350670,19,10,167


# Modeling

In [51]:
import sys
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from pykrige.rk import RegressionKriging
from pykrige.compat import train_test_split

In [52]:
svr_model = SVR(C=0.1)
rf_model = RandomForestRegressor(n_estimators=100)
lr_model = LinearRegression(normalize=True, copy_X=True, fit_intercept=False)

models = [svr_model, rf_model, lr_model]

In [53]:
target = df_model['Atendimentos'].values
x = df_model[['Lat', 'Long']].values
df_model = df_model.drop(['Lat', 'Long', 'Data_Dia', 'Atendimentos', 'Descrição da Unidade'], axis=1)
p = df_model.values

p_train, p_test, x_train, x_test, target_train, target_test \
    = train_test_split(p, x, target, test_size=0.3, random_state=42)

for m in models:
    print('=' * 40)
    print('regression model:', m.__class__.__name__)
    m_rk = RegressionKriging(regression_model=m, n_closest_points=10)
    m_rk.fit(p_train, x_train, target_train)
    print('Regression Score: ', m_rk.regression_model.score(p_test, target_test))
    print('RK score: ', m_rk.score(p_test, x_test, target_test))

regression model: SVR
Finished learning regression model
Finished kriging residuals
Regression Score:  -0.06257524144526183
RK score:  0.8225680109747848
regression model: RandomForestRegressor
Finished learning regression model
Finished kriging residuals
Regression Score:  0.8651420500109129
RK score:  0.8693780287410684
regression model: LinearRegression
Finished learning regression model
Finished kriging residuals
Regression Score:  0.834032045552448
RK score:  0.8461909124137805
